EMBEDDINGS TO FETCH THE STORIES


In [50]:
import json
from pathlib import Path
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [51]:
import json

def load_json(file_path, type_name):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        for entry in data:
            entry["type"] = type_name
        return data

# Example: loading legends
legends = load_json("data/legends.json", "legend")
short_stories = load_json("data/short_stories.json", "short_story")

all_stories = legends + short_stories

print(f"Loaded {len(all_stories)} stories.")


Loaded 72 stories.


In [52]:
texts = [f"{s['title']}\n{s['narration']}\nLesson: {s['lesson']}" for s in all_stories]
metadatas = [{"title": s["title"], "type": s["type"], "lesson": s["lesson"]} for s in all_stories]


In [54]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Initialize Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Ensure all metadata values are strings, numbers, bool, or None
def sanitize_metadata(meta):
    sanitized = {}
    for k, v in meta.items():
        if isinstance(v, list):
            sanitized[k] = " | ".join(str(i) for i in v)  # convert list to string
        elif isinstance(v, dict):
            sanitized[k] = str(v)  # convert dict to string
        else:
            sanitized[k] = v
    return sanitized

sanitized_metadatas = [sanitize_metadata(m) for m in metadatas]

# Create or load the Chroma vectorstore
vectorstore = Chroma.from_texts(
    texts,
    embeddings,
    metadatas=sanitized_metadatas,
    persist_directory="story_db"
)

print("✅ Embeddings created & saved safely in 'story_db/'.")


✅ Embeddings created & saved safely in 'story_db/'.


In [57]:
query = "a story about betrayal"
results = vectorstore.similarity_search(query, k=2)

for r in results:
    print(f"📖 {r.metadata['title']} ({r.metadata['type']})")
    print(r.page_content[:300], "...\n")


📖 The Goat and the Goatherd (short_story)
The Goat and the Goatherd
A Goatherd drove his flock into a cave during a storm. Inside, he found some wild Goats and shut them in with his own. He fed the wild Goats better, hoping they would stay. When the storm ended, they ran away, saying, “If you betray your own goats for us, how can we trust y ...

📖 The Ass in the Lion’s Skin (short_story)
The Ass in the Lion’s Skin
A foolish Ass once found a Lion’s skin and put it on. Proud of his disguise, he wandered through the forest, frightening all the animals. But when he brayed loudly, the Fox laughed and said, “If you had held your tongue, you might have frightened me too. But your voice bet ...

